In [3]:
import os
import pandas as pd
import sys
import json
from google.protobuf.json_format import Parse

sys.path.insert(0, 'compiled_protobufs')
from taskmap_pb2 import TaskMap

qrels_folder = os.path.join(os.getcwd(), 'measurements', 'judgments')
run_cooking_folder = os.path.join(os.getcwd(), 'measurements', 'cooking', 'run_files')
run_diy_folder = os.path.join(os.getcwd(), 'measurements', 'diy', 'run_files')


In [34]:
# ## save qrels files

# with open(qrels_path, "w") as f:
#     pd_annotations = pd.read_csv(annotations_path)
#     lines = []
#     # print(pd_annotations.head())
#     for idx, annotation in pd_annotations.iterrows():
#         q_id, doc_id, score = annotation["query-id"], annotation["doc-id"], annotation["relevance"]
#         lines.append(f'{q_id} Q0 {doc_id} {score}\n')
#     lines[-1] = lines[-1].replace("\n","")
#     f.writelines(lines)
    

In [54]:
# run_path = os.path.join(os.getcwd(), 'datasets', 'qrles')
# taskmap_cooking_index_path = os.path.join(os.getcwd(), "indexes", "food", "system_index_sparse")


In [69]:

# from pygaggle.rerank.base import Query, Text, hits_to_texts
# from pygaggle.rerank.transformer import MonoT5

# config = [
#     "bm25",
#     "bm25+rm3",
#     "bm25+t5",
#     "bm25+rm3+t5",
# ]

In [1]:


# cooking_queries = pd.read_csv(queries_path).iloc[:10]
# reranker =  MonoT5()

# def get_searcher(search_model):
#     if search_model == "bm25" or search_model == "bm25+t5":
#         searcher = LuceneSearcher(index_dir=taskmap_cooking_index_path)
#         searcher.set_bm25(b=0.4, k1=0.9)
#     if search_model == "bm25+rm3" or search_model == "bm25+rm3+t5":
#         searcher = LuceneSearcher(index_dir=taskmap_cooking_index_path)
#         searcher.set_bm25(b=0.4, k1=0.9)
#         searcher.set_rm3(fb_terms=10, fb_docs=10, original_query_weight=0.5)
#     return searcher

# for model in config:
#     lines = []
#     for idx, query in cooking_queries.iterrows():
#         print(f"Initialize searcher {model}")
#         searcher = get_searcher(model)
#         hits = searcher.search(q=query["target query"], k=50)
#         if "t5" in model:
#             hits = reranker.rerank(Query(query["target query"]), hits_to_texts(hits))
#         for rank, hit in enumerate(hits):
#             if type(hit) == Text:
#                 doc_json = json.loads(hit.text)
#             else:
#                 doc_json = json.loads(hit.raw)
#             taskmap_json = doc_json['recipe_document_json']
#             taskmap = Parse(json.dumps(taskmap_json), TaskMap())
#             doc_id = taskmap.taskmap_id
#             line = f'query-{idx} Q0 {doc_id} {rank+1} {hit.score} bm25\n'
#             lines.append(line)
#     lines[-1] = lines[-1].replace("\n","")

#     with open(os.path.join(run_path, model+".run"), "w") as f:
#         f.writelines(lines)
        



In [5]:
import ir_measures
from ir_measures import *

# qrles = ir_measures.read_trec_qrels('qrels/qrls.qrles')
# run = ir_measures.read_trec_run('qrels.run')

# for model in config:
#     print(os.path.join(run_path, model+".run"))
#     qrles = ir_measures.read_trec_qrels(qrels_path)
#     run = ir_measures.read_trec_run(os.path.join(run_path, model+".run"))
#     accuracy = ir_measures.calc_aggregate([nDCG@3, Precision@3, Recall@3], qrles, run)
#     print(f"{model}: {accuracy}")

def get_accuracy(run_filepath, qrles_filepath):
    run = ir_measures.read_trec_run(run_filepath)
    qrles = ir_measures.read_trec_qrels(qrles_filepath)
    accuracy = ir_measures.calc_aggregate([nDCG@10, Precision@3, Precision@5, Precision@10, MAP], qrles, run)
    return accuracy
    
#     accuracy = ir_measures.calc_aggregate([nDCG@3, Precision@3, Recall@3], qrles, run)

In [5]:
cooking_runs = os.listdir(run_cooking_folder)
cooking_qrles = os.path.join(qrels_folder, "cooking.qrles")

for searcher in cooking_runs:
    run = os.path.join(run_cooking_folder, searcher)
    searcher_name = searcher[:-4].lower()
    # print(searcher_name, run)
    accuracy = get_accuracy(run, cooking_qrles)
    print(searcher_name, accuracy)
    

bm25+rm3 {P@3: nan, nDCG@10: nan, P@10: nan, P@5: nan, AP: nan}
marqo {P@3: nan, nDCG@10: nan, P@10: nan, P@5: nan, AP: nan}
marqo-target {P@3: nan, nDCG@10: nan, P@10: nan, P@5: nan, AP: nan}
bm25+rm3+t5 {P@3: nan, nDCG@10: nan, P@10: nan, P@5: nan, AP: nan}
ance {P@3: nan, nDCG@10: nan, P@10: nan, P@5: nan, AP: nan}
bm25 {P@3: nan, nDCG@10: nan, P@10: nan, P@5: nan, AP: nan}
bm25+t5 {P@3: nan, nDCG@10: nan, P@10: nan, P@5: nan, AP: nan}
tct-colbertv2 {P@3: nan, nDCG@10: nan, P@10: nan, P@5: nan, AP: nan}


In [7]:
diy_runs = os.listdir(run_diy_folder)
diy_qrles = os.path.join(qrels_folder, "diy.qrles")

results = {}
for searcher in sorted(diy_runs):
    run = os.path.join(run_diy_folder, searcher)
    searcher_name = searcher[:-4].lower()
    # print(searcher_name, run)
    accuracy = get_accuracy(run, diy_qrles)
    results[searcher_name] = accuracy
    # print(searcher_name, accuracy)
    # print(type(accuracy))
    
df = pd.DataFrame(results).transpose()
df
    

,P@3,P@5,P@10,AP,nDCG@10
tct-colbertv2-normal,0.806667,0.726,0.576,0.515468,0.649280
tct-colbertv2-target,0.540000,0.514,0.456,0.363448,0.467546
tct-colbertv2,0.746667,0.714,0.572,0.500714,0.632030
ance-normal,0.750000,0.662,0.504,0.421227,0.595871
ance-target,0.650000,0.556,0.455,0.333379,0.500527
ance,0.716667,0.646,0.499,0.408159,0.574389
bm25+rm3+t5,0.810000,0.782,0.701,0.613461,0.721481
bm25+rm3,0.743333,0.702,0.617,0.543364,0.617084
bm25+t5,0.790000,0.768,0.684,0.598473,0.710791
bm25,0.730000,0.692,0.586,0.521407,0.596877
